In [1]:
import pandas as pd
import numpy as np

In [2]:
nodes = pd.read_csv("../data/test/demand.csv", index_col="node")
nodes.rename(columns={"distance to depot": "d0"}, inplace=True)
nodes

,d0,demand
node,,
0,0,0
1,25,4
2,43,6
3,57,5
4,43,4
5,61,7
6,29,3
7,41,5
8,48,4


In [3]:
pw = pd.read_csv("../data/test/pairwise.csv", index_col=0)
pw.columns = pw.columns.astype(int)  # Ensure columns are integers
pw.index = pw.index.astype(int)  # Ensure the indices are integers
pw

,1,2,3,4,5,6,7,8,9
1,0,29,34,43,68,49,66,72,91
2,29,0,52,72,96,72,81,89,114
3,34,52,0,45,71,71,95,99,108
4,43,72,45,0,27,36,65,65,65
5,68,96,71,27,0,40,66,62,46
6,49,72,71,36,40,0,31,31,43
7,66,81,95,65,66,31,0,11,46
8,72,89,99,65,62,31,11,0,36
9,91,114,108,65,46,43,46,36,0


In [4]:
savings = []
for r in pw.index:
    for c in pw.columns:
        if r > c:
            saving = nodes["d0"][r] + nodes["d0"][c] - pw.loc[r, c]
            savings.append((r, c, int(saving)))

In [5]:
savings.sort(key=lambda x: x[2], reverse=True) # sort by saving in descending order
savings

[(9, 5, 86),
 (9, 8, 83),
 (8, 7, 78),
 (5, 4, 77),
 (9, 7, 66),
 (9, 6, 57),
 (4, 3, 55),
 (6, 5, 50),
 (9, 4, 49),
 (3, 1, 48),
 (3, 2, 48),
 (5, 3, 47),
 (8, 5, 47),
 (8, 6, 46),
 (2, 1, 39),
 (7, 6, 39),
 (6, 4, 36),
 (7, 5, 36),
 (8, 4, 26),
 (4, 1, 25),
 (9, 3, 20),
 (7, 4, 19),
 (5, 1, 18),
 (6, 3, 15),
 (4, 2, 14),
 (5, 2, 8),
 (8, 3, 6),
 (6, 1, 5),
 (9, 1, 5),
 (7, 2, 3),
 (7, 3, 3),
 (8, 2, 2),
 (8, 1, 1),
 (6, 2, 0),
 (7, 1, 0),
 (9, 2, 0)]

In [6]:
depot = 0

In [47]:
routes = {i: [] for i in range(1, len(nodes))}
routes

{1: [], 2: [], 3: [], 4: [], 5: [], 6: [], 7: [], 8: [], 9: []}

In [48]:
def interior(node):
    route = routes[node]
    i = route.index(node)
    if i == 0 or i == len(route) - 1: # adjacent to depot -> not interior
        return False
    return True

In [49]:
for i, j, saving in savings:
    if not routes[i] and not routes[j]:
        new_route = [i, j]
        routes[i] = routes[j] = new_route
    elif not routes[i]:
        if not interior(j):
            route_j = routes[j]
            index_j = route_j.index(j)
            route_j.insert(index_j, i)
            routes[i] = route_j
            
    elif not routes[j]:
        if not interior(i):
            route_i = routes[i]
            index_i = route_i.index(i)
            route_i.insert(index_i, j)
            routes[j] = route_i
    else:
        if not interior(i) and not interior(j) and routes[i] != routes[j]:
            route_i, route_j = routes[i], routes[j]
            index_i, index_j = route_i.index(i), route_j.index(j)
            if index_i == 0: # move node i to tail
                route_i = route_i[::-1]
            if index_j == len(route_j) - 1: # move node j to head
                route_j = route_j[::-1]
            # print(i, j, route_i, route_j)
            # break
            merged_route = route_i + route_j
            routes[i] = routes[j] = merged_route
            print(merged_route)
            for node in merged_route:
                routes[node] = merged_route

[7, 8, 9, 4, 6, 5, 1, 3, 2]


In [50]:
routes

{1: [7, 8, 9, 4, 6, 5, 1, 3, 2],
 2: [7, 8, 9, 4, 6, 5, 1, 3, 2],
 3: [7, 8, 9, 4, 6, 5, 1, 3, 2],
 4: [7, 8, 9, 4, 6, 5, 1, 3, 2],
 5: [7, 8, 9, 4, 6, 5, 1, 3, 2],
 6: [7, 8, 9, 4, 6, 5, 1, 3, 2],
 7: [7, 8, 9, 4, 6, 5, 1, 3, 2],
 8: [7, 8, 9, 4, 6, 5, 1, 3, 2],
 9: [7, 8, 9, 4, 6, 5, 1, 3, 2]}